# Aggregate All QC Filters

There are 5 main QC filters:
 - variant missingness
 - individual missingness
 - variant hwe
 - variant read depth
 - variant allelic balance

Each one has it's own threshold & each one was applied individually to build a mask.

This notebook aggregates them & demonstrates how each one refines the UKBB Exome data into the format we want to use it in.

# Imports & Globals

In [51]:
import pandas as pd
import numpy as np
from collections import defaultdict

DATA_OUT = "/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/"
DATA_IN = "/mnt/speliotes-lab/Projects/UK_ATLAS/GENODATA/Exomechip_10_13_2022/"
rap_output_files = "{}rap_runs/{}/".format(DATA_OUT,"{}")
sample_missing_files = "{}allele_counts/chr{}/var_counts.smiss".format(DATA_OUT,"{}")
variant_missing_files = "{}allele_counts/chr{}/var_counts.vmiss".format(DATA_OUT,"{}")
hwe_files = "{}allele_hwe/chr{}/var_hwe.hardy".format(DATA_OUT,"{}")
CHROMOS = list(range(1,22+1))
GROUPS = ["group_{}".format(i) for i in range(1,10+1)]

from glob import glob

# Read Depth & Allelic Balance

Read depth & allelic balance were run on the UK BioBank (UKBB) Research Access Portal (RAP). Early termination was used on them so values of 0.0 mean that variant had such poor read depth that actual was not calculated. Furthermore since allelic balance was 1st sample to qualify, once a sample qualified allelic balance calculation was stopped. Third allelic balance was run after read depth so if read depth was insufficient allelic balance calculation was skipped resulting in value of 0.0.

In [2]:
new_data_list = []
for dname in glob(rap_output_files[:-3]+"*"):
    if "." not in dname:
        print(dname)
        for fname in glob(dname+"/*"):
            with open(fname, "r") as f:
                for line in f:
                    line = line.strip().split()
                    for snp in line[0].split(";"):
                        new_data_list.append({"snpid":snp,
                                              "rd_10":float(line[1]),
                                              "ab":float(line[2])})
new_data_df = pd.DataFrame(new_data_list)
print(new_data_df.shape)
#new_data_df.head()

# 10%_read_depth
indel_mask = np.array([bool(snp[-2] != "_" or snp[-4] != "_") for snp in new_data_df.snpid])
rd_mask = np.array([rd >= (10 if indel else 7) for indel,rd in zip(indel_mask,new_data_df.rd_10)])
ab_mask = np.array([ab >= (0.2 if indel else 0.15) for indel,ab in zip(indel_mask,new_data_df.ab)])
rd10_mask = rd_mask & ab_mask
print(np.sum(rd10_mask),len(rd10_mask),np.sum(rd10_mask)/len(rd10_mask))

/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_9
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_10
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_4
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_2
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_8
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_7
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_1
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/local_vcfs
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_5
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_3
/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/rap_runs/group_6
(26388600, 3)
23349895 26388

# Variant & Individual Missingness

Missingness was calculated via Plink2 locally in the same run on the original bgen files downloaded directly from the RAP.

Bgen files were assumed to be 'ref-first'

In [9]:
# variant_missingness
variant_missingness_df = pd.DataFrame()
for chrom in CHROMOS:
    tmp_df = pd.read_table(variant_missing_files.format(chrom))
    if variant_missingness_df.empty:
        variant_missingness_df = tmp_df
    else:
        variant_missingness_df = pd.concat([variant_missingness_df,tmp_df])
print(variant_missingness_df.shape)
vmiss_mask = np.array([f <= 0.1 for f in variant_missingness_df.F_MISS])

print(np.sum(vmiss_mask),len(vmiss_mask),np.sum(vmiss_mask)/len(vmiss_mask))

(26388327, 5)
25989060 26388327 0.9848695599383773


In [17]:
# individual_missingness
empty_tuple = lambda : (0,0)
ind_to_cnts = defaultdict(empty_tuple)
for chrom in CHROMOS:
    with open(sample_missing_files.format(chrom),"r") as f:
        for i,line in enumerate(f):
            if i > 0: # don't process header
                fid,iid,m_cnt,o_cnt,f_miss = map(float,line.strip().split())
                ind_to_cnts[iid] = (ind_to_cnts[iid][0]+m_cnt,ind_to_cnts[iid][1]+o_cnt)
sample_missingness_df = pd.DataFrame([{"IID":iid,"MISSING_CT":ind_to_cnts[iid][0],"OBS_CT":ind_to_cnts[iid][1]} for iid in ind_to_cnts])
sample_missingness_df["F_MISS"] = sample_missingness_df.MISSING_CT/sample_missingness_df.OBS_CT
print(sample_missingness_df.shape)
smiss_mask = np.array([f <= 0.1 for f in sample_missingness_df.F_MISS])

print(np.sum(smiss_mask),len(smiss_mask),np.sum(smiss_mask)/len(smiss_mask))

(469835, 4)
469835 469835 1.0


# Hardy Weinberg Equilibrium

HWE was calculated locally via Plink2. Bgen data was processed for this analysis.

In [52]:
# hwe equilibrium
hwe_df = pd.DataFrame()
for chrom in CHROMOS:
    tmp_df = pd.read_table(hwe_files.format(chrom))
    if hwe_df.empty:
        hwe_df = tmp_df
    else:
        hwe_df = pd.concat([hwe_df,tmp_df])
print(hwe_df.shape)
hwe_mask = np.array([p > 1e-15 for p in hwe_df.P])

print(np.sum(hwe_mask),len(hwe_mask),np.sum(hwe_mask)/len(hwe_mask))

(26388327, 10)
26150930 26388327 0.9910037116032403


# Combining all the filters

## Missing Read Depth for bgen variants

So not every variant found in the bgen file is found in the vcf file.

In fact 331 variants are unique to the bgen format. To invesitigate the reason behind this I decided to study these vcf free variants & noticed they were all InDels. Furthermore, half 174 had high missingness in the bgen file so most of these InDels should have been omitted.

That leaves 157 variants without Read Depth

In [29]:
bgen_variants = set(variant_missingness_df.ID)
def convert_vcf_snpid_to_bgen(snpid):
    return snpid[3:].replace("_",":")
vcf_variants = set(map(convert_vcf_snpid_to_bgen,new_data_df.snpid))
print(len(bgen_variants),len(vcf_variants))
print(len(bgen_variants & vcf_variants))

26388327 26388600
26387996


In [31]:
unmapped_bgen_var = bgen_variants - vcf_variants
print(len(unmapped_bgen_var))

331


In [36]:
var_to_miss_filt = {var:filt for var,filt in zip(variant_missingness_df.ID,vmiss_mask)}
var_to_rdab_filt = {convert_vcf_snpid_to_bgen(var):filt for var,filt in zip(new_data_df.snpid,rd10_mask)}
print(len(var_to_miss_filt),len(var_to_rdab_filt))

26388327 26388600


In [39]:
shoulda_kept = 0
for var in unmapped_bgen_var:
    if var_to_miss_filt[var]:
        shoulda_kept += 1
print(shoulda_kept)

157


Normally this is where things would end, but since these are all InDels ... we can make use of the 90pct high quality file that the UKBB provided earlier.

It is a file with a list of variants of which 90% of the read depths are >= 10 which just so happens to be the cutoff threshold for InDels for our threshold.

In [46]:
hq_var = set()
with open("{}helper_files/ukb23158_500k_OQFE.90pct10dp_qc_variants.txt".format(DATA_IN),"r") as f:
    for i,line in enumerate(f):
        hq_var.add(line.strip())
print(len(hq_var))

5797991


In [48]:
good_unmapped = set()
for var in unmapped_bgen_var:
    if var in hq_var:
        good_unmapped.add(var)
print(len(good_unmapped))

268


We would still need the allelic balance for these variants, but since they are in the High Quality file provided by UKBB I will assume that they pass the vcf filter threshold which includes allelic balance.

I do not assume that they automatically pass the missingness threshold. Any variants not in this file ... I think I can assume will not pass the Read Depth and Allelic balance thresholds.

In [50]:
bgen_rdab_mask = []
for var in variant_missingness_df.ID:
    if var in var_to_rdab_filt:
        bgen_rdab_mask.append(var_to_rdab_filt[var])
    else:
        bgen_rdab_mask.append(var in good_unmapped)
bgen_rdab_mask = np.array(bgen_rdab_mask)
print(np.sum(bgen_rdab_mask),len(bgen_rdab_mask),np.sum(bgen_rdab_mask)/len(bgen_rdab_mask))

23349853 26388327 0.8848553756363562


Okay so following this protocol we get a 100% match with the variants from the VCF filter and the variants across the bgen filters.

Proceed to the final merge now.

## Final Merge

So from each of the 4 indpendent masks:
- variant missingness --> keep 98% of variants
- hwe --> keep 99% of variants
- read depth and allelic balance --> keep ~88% of variants
- individual missingness --> 100% of individuals

The individual mask is not necessary (🙏) so I will ignore it & apply the masks in the following order:
- no masks: 26,388,327 variants
- variant missingness mask: 25,989,060 variants
- HWE mask: 25,786,269 variants
- Read Depth & Allelic Balance mask: 22,946,846 variants

In [55]:
all_variants = set(variant_missingness_df.ID)
print(len(all_variants))

26388327


In [56]:
vm_variants = set(variant_missingness_df[vmiss_mask].ID)
print(len(all_variants & vm_variants))

25989060


In [57]:
hwe_variants = set(hwe_df[hwe_mask].ID)
print(len(all_variants & vm_variants & hwe_variants))

25786269


In [58]:
rdab_variants = set(variant_missingness_df[bgen_rdab_mask].ID)
print(len(all_variants & vm_variants & hwe_variants & rdab_variants))

22946846


In [59]:
final_good_clean_variants = all_variants & vm_variants & hwe_variants & rdab_variants

So now all that's left to do is to write these variants to 22 text files.
- one for each chromosome
- one variant per line

Then run the final Plink bgen filter.

In [60]:
22946846/26388327

0.8695832062411535